In [1]:
#!/bin/bash
!kaggle datasets download omkargurav/face-mask-dataset

Dataset URL: https://www.kaggle.com/datasets/omkargurav/face-mask-dataset
License(s): unknown
100%|███████████████████████████████████████▉| 163M/163M [00:26<00:00, 9.84MB/s]
100%|████████████████████████████████████████| 163M/163M [00:26<00:00, 6.45MB/s]


### Importing Neccessary Libraries

In [6]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator

### Split Data into Train & Test Dataset

In [16]:
import os
import shutil
from sklearn.model_selection import train_test_split

# Paths for original dataset
with_mask_dir = './face-mask-dataset/data/with_mask/'
without_mask_dir = './face-mask-dataset/data/without_mask/'

# Define train/test directories
train_dir = './face-mask-dataset/data/train'
test_dir = './face-mask-dataset/data/test'

# Create train/test directories if they don't exist
os.makedirs(os.path.join(train_dir, 'with_mask'), exist_ok=True)
os.makedirs(os.path.join(train_dir, 'without_mask'), exist_ok=True)
os.makedirs(os.path.join(test_dir, 'with_mask'), exist_ok=True)
os.makedirs(os.path.join(test_dir, 'without_mask'), exist_ok=True)

# Split with_mask images
images_w_mask = os.listdir(with_mask_dir)
train_images, test_images = train_test_split(images_w_mask, test_size=0.2, random_state=1)

# Split without_mask images
images_wo_mask = os.listdir(without_mask_dir)
train_images1, test_images1 = train_test_split(images_wo_mask, test_size=0.2, random_state=1)

# Move images into train/test folders
for image in train_images:
    shutil.move(os.path.join(with_mask_dir, image), os.path.join(train_dir, 'with_mask', image))
for image in test_images:
    shutil.move(os.path.join(with_mask_dir, image), os.path.join(test_dir, 'with_mask', image))

for image in train_images1:
    shutil.move(os.path.join(without_mask_dir, image), os.path.join(train_dir, 'without_mask', image))
for image in test_images1:
    shutil.move(os.path.join(without_mask_dir, image), os.path.join(test_dir, 'without_mask', image))

print("Images have been successfully moved to train and test directories.")


Images have been successfully moved to train and test directories.


### Processing Images

In [17]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=15,
    zoom_range=0.2,
    width_shift_range=0.2,
    height_shift_range=0.2
)

test_datagen = ImageDataGenerator(
    rescale=1./255
)

train_generator = train_datagen.flow_from_directory(
    train_dir,          
    target_size=(128, 128),
    batch_size=32,
    class_mode='binary',  
)

# Load testing data
test_generator = test_datagen.flow_from_directory(
    test_dir,            
    target_size=(128, 128),
    batch_size=32,
    class_mode='binary',  
)

Found 6042 images belonging to 2 classes.
Found 1511 images belonging to 2 classes.


### Building CNN Model

In [28]:
model = keras.Sequential([
    keras.layers.Input(shape=(128,128,3)),
    keras.layers.Conv2D(32, kernel_size=(3,3), activation='relu'),
    keras.layers.MaxPooling2D(pool_size=(2,2)),

    keras.layers.Conv2D(64, kernel_size=(3,3), activation='relu'),
    keras.layers.MaxPooling2D(pool_size=(2,2)),

    keras.layers.Flatten(),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dropout(0.2),

    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dropout(0.2),

    keras.layers.Dense(1, activation='sigmoid')
    
    
])
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

In [32]:
model.fit(train_generator, epochs=10)

Epoch 1/10
189/189 ━━━━━━━━━━━━━━━━━━━━ 21s 109ms/step - accuracy: 0.8890 - loss: 0.2725
Epoch 2/10
189/189 ━━━━━━━━━━━━━━━━━━━━ 20s 103ms/step - accuracy: 0.9018 - loss: 0.2476
Epoch 3/10
189/189 ━━━━━━━━━━━━━━━━━━━━ 20s 108ms/step - accuracy: 0.9094 - loss: 0.2347
Epoch 4/10
189/189 ━━━━━━━━━━━━━━━━━━━━ 19s 102ms/step - accuracy: 0.9033 - loss: 0.2279
Epoch 5/10
189/189 ━━━━━━━━━━━━━━━━━━━━ 20s 107ms/step - accuracy: 0.9300 - loss: 0.2050
Epoch 6/10
189/189 ━━━━━━━━━━━━━━━━━━━━ 20s 105ms/step - accuracy: 0.9289 - loss: 0.1911
Epoch 7/10
189/189 ━━━━━━━━━━━━━━━━━━━━ 20s 103ms/step - accuracy: 0.9187 - loss: 0.2094
Epoch 8/10
189/189 ━━━━━━━━━━━━━━━━━━━━ 20s 104ms/step - accuracy: 0.9374 - loss: 0.1644
Epoch 9/10
189/189 ━━━━━━━━━━━━━━━━━━━━ 21s 113ms/step - accuracy: 0.9351 - loss: 0.1772
Epoch 10/10
189/189 ━━━━━━━━━━━━━━━━━━━━ 20s 106ms/step - accuracy: 0.9386 - loss: 0.1697


### Evaluating Model & Saving Model

In [34]:
model.evaluate(test_generator)

48/48 ━━━━━━━━━━━━━━━━━━━━ 2s 46ms/step - accuracy: 0.9517 - loss: 0.1217


[0.11477652937173843, 0.9530112743377686]

In [38]:
model.save("path_to_model/Face_Mask_Prediction_model.keras")

### Building Predictive System

In [40]:
from tensorflow.keras.models import load_model

model = load_model('path_to_model/Face_Mask_Prediction_model.keras')

In [62]:
from tensorflow.keras.preprocessing import image
import numpy as np

# Load and preprocess the image
input_img = 'face-mask-dataset/data/test/with_mask/with_mask_1006.jpg'
img = image.load_img(input_img)

if img is None:
    print("Error: Image could not be loaded.")
else:
    print("Image loaded successfully.")

img = img.resize((128, 128))

img = image.img_to_array(img)

img = img / 255.0 

img = np.expand_dims(img, axis=0)


# Make the prediction
result = model.predict(img)

# Interpret the result
if result <0.5:
    print('Person is Wearing Mask')
else:
    print('Person is Not Wearing Mask')




Image loaded successfully.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
Person is Wearing Mask
